## Initialize

In [1]:
from dummy_spark import SparkContext
# from pyspark import SparkContext

In [2]:
from sympy import *
from agp_fermi import *

In [3]:
ctx = SparkContext()

In [4]:
dr = AGPFermi(ctx)

In [5]:
names = dr.names

In [6]:
p, q , r, s  = symbols('p q r s')

## Creation / annihilation operators

In [7]:
c_p = dr.sum(names.c_[p])

In [8]:
cdag_p = dr.sum(names.c_dag[q])

In [9]:
dr.simplify(c_p*cdag_p + cdag_p*c_p).display()

<IPython.core.display.Math object>

In [11]:
(dr.dagger(c_p)).display()

<IPython.core.display.Math object>

## Pairing operators

In [10]:
P_p = dr.sum( dr.P[p] )
Pdag_q = dr.sum( dr.Pdag[q] )

## Spin range sum

In [12]:
dr.sum(names.c_[p,UP]).display()

<IPython.core.display.Math object>

In [13]:
(dr.sum((s,dr.spin_range),names.c_[p,s])).simplify().display()

<IPython.core.display.Math object>

## Spin flip su(2)

In [23]:
Sp = dr.sum(names.J_p[p])

In [24]:
Sp.display()

<IPython.core.display.Math object>